In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import gc
import numpy as np
import pandas as pd
import tensorflow as tf

from collections import defaultdict
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, balanced_accuracy_score, roc_auc_score, average_precision_score, fbeta_score, matthews_corrcoef
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, LayerNormalization, Input, Conv1D, Lambda, Flatten

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


def import_data(file):
    """create a dataframe and optimize its memory usage"""
    df = pd.read_csv(file, parse_dates=True, keep_date_col=True)
    df = reduce_mem_usage(df)
    return df

In [4]:
def create_convnext_like_model(input_dim, num_classes, sliding_window_size=3):
    # Input layer
    input_layer = Input(shape=(input_dim,))
    
    # Reshape input to be compatible with Conv1D using Lambda layer
    reshaped_input = Lambda(lambda x: tf.expand_dims(x, axis=-1))(input_layer)  # Shape: (batch_size, input_dim, 1)
    x = reshaped_input
    
    # Simulate ConvNeXt block using Conv1D layers
    x = Conv1D(filters=int(max(16, input_dim * 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 2)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 4)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    x = Conv1D(filters=int(max(16, input_dim // 8)), kernel_size=sliding_window_size, padding='same', activation='gelu', strides=1)(x)
    x = LayerNormalization()(x)
    
    # Flatten the output of the final Conv1D layer
    x = Flatten()(x)
    
    # Global pooling and classification head
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    output_layer = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [5]:
def run_model(train_data_path, test_data_path, is_string_labels = False, label_mapping = None):

    # Initialize the one-hot encoder for the target
    encoder = OneHotEncoder(sparse_output=False)

    # Load and Prepare Training Data
    train_data = pd.read_csv(train_data_path)
    train_data = train_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        train_data['label'] = train_data['label'].map(label_mapping)
    train_X = train_data.drop(columns=['label']).values
    train_y = train_data['label'].values
    train_y = encoder.fit_transform(train_y.reshape(-1, 1))
    del train_data
    gc.collect()

    # Load and Prepare Test Data (this will not be used in training)
    test_data = pd.read_csv(test_data_path)
    test_data = test_data.sample(frac=1).reset_index(drop=True)  # Shuffle
    if (is_string_labels):
        test_data['label'] = test_data['label'].map(label_mapping)
    test_X = test_data.drop(columns=['label']).values
    test_y = test_data['label'].values
    test_y = encoder.transform(test_y.reshape(-1, 1))
    del test_data
    gc.collect()

    # EarlyStopping Callback (optional, to avoid overfitting)
    early_stopping = EarlyStopping(monitor='loss', patience=10, restore_best_weights=True)

    # Number of runs for averaging results
    num_runs = 50

    # Initialize storage for metrics
    metrics_storage = defaultdict(list)

    # Train the Model with Validation Split N tines for more accurate metrics
    #print("Verbose output only for first run...")
    verbose_run = 0
    for run in range(num_runs):
        
        # Model is defined separately in each run, since the random combination layers
        # must be randomly initialized each time. Otherwise, the "random" indices stay the same
        # throughout all runs
        model = create_convnext_like_model(train_X.shape[1], test_y.shape[1])

        print(f"Run {run + 1}/{num_runs} started...")
        history = model.fit(
            train_X, train_y, 
            epochs=100, 
            batch_size=int(train_X.shape[0] * 0.01),
            callbacks=[early_stopping],
            verbose=verbose_run
        )
        verbose_run = 0 # Suppress detailed output for multiple runs

        test_loss, test_acc = model.evaluate(test_X, test_y, verbose=0)
        y_pred = model.predict(test_X, verbose=0)
        y_pred_classes = y_pred.argmax(axis=1)
        y_true_classes = test_y.argmax(axis=1)
        del model
        gc.collect()

        # Compute metrics
        balanced_acc = balanced_accuracy_score(y_true_classes, y_pred_classes)
        roc_auc = roc_auc_score(test_y, y_pred, multi_class='ovr')  # `test_y` is fine here for AUC
        pr_auc = average_precision_score(test_y, y_pred, average='weighted')
        f2 = fbeta_score(y_true_classes, y_pred_classes, beta=2, average='weighted')
        mcc = matthews_corrcoef(y_true_classes, y_pred_classes)

        # Store metrics
        metrics_storage['test_loss'].append(test_loss)
        metrics_storage['test_accuracy'].append(test_acc)
        metrics_storage['balanced_accuracy'].append(balanced_acc)
        metrics_storage['roc_auc'].append(roc_auc)
        metrics_storage['pr_auc'].append(pr_auc)
        metrics_storage['f2'].append(f2)
        metrics_storage['mcc'].append(mcc)

        # Store classification report metrics
        report = classification_report(y_true_classes, y_pred_classes, output_dict=True)
        for label, values in report.items():
            # Check if the value is a dictionary (e.g., 'precision', 'recall', 'f1-score')
            if isinstance(values, dict):
                for metric, value in values.items():
                    metrics_storage[f"{label}_{metric}"].append(value)
            else:
                # Handle scalar values (like 'accuracy')
                metrics_storage[label].append(values)

        # Average the metrics over all successful runs
        print(f"\nAggregated Metrics for {run+1} runs:")
        for metric, values in metrics_storage.items():
            avg_value = np.mean(values)
            print(f"{metric}: {avg_value:.4f}")

    gc.collect()
        

In [ ]:
run_model("/kaggle/input/ma-bsmote/shuttle_bsmote.csv", "/kaggle/input/ma-datasets/shuttle_test.csv", is_string_labels = False)

Run 1/50 started...

Aggregated Metrics for 1 runs:
test_loss: 0.0041
test_accuracy: 0.9994
balanced_accuracy: 0.9281
roc_auc: 1.0000
pr_auc: 0.9997
f2: 0.9994
mcc: 0.9983
0_precision: 1.0000
0_recall: 0.9997
0_f1-score: 0.9998
0_support: 9117.0000
1_precision: 0.9091
1_recall: 1.0000
1_f1-score: 0.9524
1_support: 10.0000
2_precision: 0.8718
2_recall: 1.0000
2_f1-score: 0.9315
2_support: 34.0000
3_precision: 1.0000
3_recall: 0.9989
3_f1-score: 0.9994
3_support: 1781.0000
4_precision: 0.9985
4_recall: 0.9985
4_f1-score: 0.9985
4_support: 653.0000
5_precision: 1.0000
5_recall: 0.5000
5_f1-score: 0.6667
5_support: 2.0000
6_precision: 1.0000
6_recall: 1.0000
6_f1-score: 1.0000
6_support: 3.0000
accuracy: 0.9994
macro avg_precision: 0.9685
macro avg_recall: 0.9281
macro avg_f1-score: 0.9355
macro avg_support: 11600.0000
weighted avg_precision: 0.9995
weighted avg_recall: 0.9994
weighted avg_f1-score: 0.9994
weighted avg_support: 11600.0000
Run 2/50 started...

Aggregated Metrics for 2 runs:

In [ ]:
run_model("/kaggle/input/ma-bsmote/covtype_bsmote.csv", "/kaggle/input/ma-datasets/covtype_test.csv", is_string_labels = False)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_loss: 0.2219
test_accuracy: 0.9138
balanced_accuracy: 0.8968
roc_auc: 0.9933
pr_auc: 0.9719
f2: 0.9138
mcc: 0.8619
0_precision: 0.9248
0_recall: 0.8990
0_f1-score: 0.9117
0_support: 42368.0000
1_precision: 0.9216
1_recall: 0.9332
1_f1-score: 0.9274
1_support: 56661.0000
2_precision: 0.9198
2_recall: 0.8656
2_f1-score: 0.8919
2_support: 7151.0000
3_precision: 0.7080
3_recall: 0.9144
3_f1-score: 0.7981
3_support: 549.0000
4_precision: 0.7800
4_recall: 0.8457
4_f1-score: 0.8115
4_support: 1899.0000
5_precision: 0.7786
5_recall: 0.8578
5_f1-score: 0.8163
5_support: 3473.0000
6_precision: 0.9128
6_recall: 0.9620
6_f1-score: 0.9367
6_support: 4102.0000
accuracy: 0.9138
macro avg_precision: 0.8494
macro avg_recall: 0.8968
macro avg_f1-score: 0.8705
macro avg_support: 116203.0000
weighted avg_precision: 0.9148
weighted avg_recall: 0.9138
weighted avg_f1-score: 0.9140
weighted avg_support: 116203.0000
Run 2/50 started...



Aggregated Metrics for 2 runs:
test_loss: 0.4282
test_accuracy: 0.8223
balanced_accuracy: 0.7319
roc_auc: 0.9691
pr_auc: 0.8827
f2: 0.8205
mcc: 0.7102
0_precision: 0.8351
0_recall: 0.8028
0_f1-score: 0.8186
0_support: 42368.0000
1_precision: 0.8369
1_recall: 0.8821
1_f1-score: 0.8585
1_support: 56661.0000
2_precision: 0.8226
2_recall: 0.7261
2_f1-score: 0.7703
2_support: 7151.0000
3_precision: 0.4520
3_recall: 0.9390
3_f1-score: 0.5619
3_support: 549.0000
4_precision: 0.5986
4_recall: 0.4866
4_f1-score: 0.5034
4_support: 1899.0000
5_precision: 0.6225
5_recall: 0.5517
5_f1-score: 0.5690
5_support: 3473.0000
6_precision: 0.8661
6_recall: 0.7349
6_f1-score: 0.7819
6_support: 4102.0000
accuracy: 0.8223
macro avg_precision: 0.7191
macro avg_recall: 0.7319
macro avg_f1-score: 0.6948
macro avg_support: 116203.0000
weighted avg_precision: 0.8243
weighted avg_recall: 0.8223
weighted avg_f1-score: 0.8200
weighted avg_support: 116203.0000
Run 3/50 started...



Aggregated Metrics for 3 runs:
test_loss: 0.4893
test_accuracy: 0.7966
balanced_accuracy: 0.6976
roc_auc: 0.9627
pr_auc: 0.8583
f2: 0.7950
mcc: 0.6694
0_precision: 0.8081
0_recall: 0.7805
0_f1-score: 0.7940
0_support: 42368.0000
1_precision: 0.8176
1_recall: 0.8579
1_f1-score: 0.8370
1_support: 56661.0000
2_precision: 0.7843
2_recall: 0.7012
2_f1-score: 0.7396
2_support: 7151.0000
3_precision: 0.3706
3_recall: 0.9393
3_f1-score: 0.4881
3_support: 549.0000
4_precision: 0.5440
4_recall: 0.4207
4_f1-score: 0.4513
4_support: 1899.0000
5_precision: 0.5854
5_recall: 0.4714
5_f1-score: 0.5082
5_support: 3473.0000
6_precision: 0.8347
6_recall: 0.7123
6_f1-score: 0.7598
6_support: 4102.0000
accuracy: 0.7966
macro avg_precision: 0.6778
macro avg_recall: 0.6976
macro avg_f1-score: 0.6540
macro avg_support: 116203.0000
weighted avg_precision: 0.7992
weighted avg_recall: 0.7966
weighted avg_f1-score: 0.7948
weighted avg_support: 116203.0000
Run 4/50 started...



Aggregated Metrics for 4 runs:
test_loss: 0.5237
test_accuracy: 0.7817
balanced_accuracy: 0.6823
roc_auc: 0.9591
pr_auc: 0.8445
f2: 0.7801
mcc: 0.6471
0_precision: 0.7880
0_recall: 0.7747
0_f1-score: 0.7811
0_support: 42368.0000
1_precision: 0.8117
1_recall: 0.8355
1_f1-score: 0.8230
1_support: 56661.0000
2_precision: 0.7516
2_recall: 0.7057
2_f1-score: 0.7259
2_support: 7151.0000
3_precision: 0.3319
3_recall: 0.9458
3_f1-score: 0.4542
3_support: 549.0000
4_precision: 0.5193
4_recall: 0.3860
4_f1-score: 0.4248
4_support: 1899.0000
5_precision: 0.5708
5_recall: 0.4057
5_f1-score: 0.4559
5_support: 3473.0000
6_precision: 0.8023
6_recall: 0.7226
6_f1-score: 0.7520
6_support: 4102.0000
accuracy: 0.7817
macro avg_precision: 0.6537
macro avg_recall: 0.6823
macro avg_f1-score: 0.6310
macro avg_support: 116203.0000
weighted avg_precision: 0.7848
weighted avg_recall: 0.7817
weighted avg_f1-score: 0.7800
weighted avg_support: 116203.0000
Run 5/50 started...


In [ ]:
labels_map = {
    'Normal': 0, 'Darknet_Audio-Streaming': 1, 'Darknet_Chat': 2, 'Darknet_File-Transfer': 3, 'Darknet_VOIP': 4,
    'Darknet_Video-Streaming': 5, 'Darknet_Email': 6, 'Darknet_Browsing': 7, 'Darknet_P2P': 8
}

run_model("/kaggle/input/ma-bsmote/darknet_bsmote.csv", "/kaggle/input/ma-datasets/darknet_test.csv", is_string_labels = True, label_mapping = labels_map)

Run 1/50 started...



Aggregated Metrics for 1 runs:
test_loss: 0.0067
test_accuracy: 0.9987
balanced_accuracy: 0.9806
roc_auc: 0.9998
pr_auc: 0.9998
f2: 0.9987
mcc: 0.9952
0_precision: 0.9997
0_recall: 0.9997
0_f1-score: 0.9997
0_support: 26862.0000
1_precision: 0.9996
1_recall: 0.9951
1_f1-score: 0.9974
1_support: 2657.0000
2_precision: 0.9880
2_recall: 0.9967
2_f1-score: 0.9923
2_support: 908.0000
3_precision: 0.9961
3_recall: 0.9904
3_f1-score: 0.9933
3_support: 522.0000
4_precision: 0.9897
4_recall: 0.9863
4_f1-score: 0.9880
4_support: 293.0000
5_precision: 0.9744
5_recall: 0.9888
5_f1-score: 0.9815
5_support: 269.0000
6_precision: 0.9915
6_recall: 1.0000
6_f1-score: 0.9957
6_support: 116.0000
7_precision: 0.8364
7_recall: 0.8679
7_f1-score: 0.8519
7_support: 53.0000
8_precision: 0.9778
8_recall: 1.0000
8_f1-score: 0.9888
8_support: 44.0000
accuracy: 0.9987
macro avg_precision: 0.9726
macro avg_recall: 0.9806
macro avg_f1-score: 0.9765
macro avg_support: 31724.0000
weighted avg_precision: 0.9987
weigh


Aggregated Metrics for 2 runs:
test_loss: 0.0216
test_accuracy: 0.9944
balanced_accuracy: 0.9527
roc_auc: 0.9996
pr_auc: 0.9982
f2: 0.9945
mcc: 0.9799
0_precision: 0.9993
0_recall: 0.9981
0_f1-score: 0.9987
0_support: 26862.0000
1_precision: 0.9971
1_recall: 0.9769
1_f1-score: 0.9868
1_support: 2657.0000
2_precision: 0.9651
2_recall: 0.9928
2_f1-score: 0.9787
2_support: 908.0000
3_precision: 0.9687
3_recall: 0.9713
3_f1-score: 0.9700
3_support: 522.0000
4_precision: 0.9442
4_recall: 0.9778
4_f1-score: 0.9604
4_support: 293.0000
5_precision: 0.8788
5_recall: 0.9442
5_f1-score: 0.9095
5_support: 269.0000
6_precision: 0.9095
6_recall: 0.9138
6_f1-score: 0.9116
6_support: 116.0000
7_precision: 0.6459
7_recall: 0.8679
7_f1-score: 0.7246
7_support: 53.0000
8_precision: 0.8931
8_recall: 0.9318
8_f1-score: 0.9120
8_support: 44.0000
accuracy: 0.9944
macro avg_precision: 0.9113
macro avg_recall: 0.9527
macro avg_f1-score: 0.9280
macro avg_support: 31724.0000
weighted avg_precision: 0.9951
weigh


Aggregated Metrics for 3 runs:
test_loss: 0.0310
test_accuracy: 0.9919
balanced_accuracy: 0.9463
roc_auc: 0.9995
pr_auc: 0.9974
f2: 0.9920
mcc: 0.9710
0_precision: 0.9992
0_recall: 0.9964
0_f1-score: 0.9978
0_support: 26862.0000
1_precision: 0.9926
1_recall: 0.9665
1_f1-score: 0.9793
1_support: 2657.0000
2_precision: 0.9466
2_recall: 0.9938
2_f1-score: 0.9693
2_support: 908.0000
3_precision: 0.9567
3_recall: 0.9655
3_f1-score: 0.9610
3_support: 522.0000
4_precision: 0.9396
4_recall: 0.9716
4_f1-score: 0.9551
4_support: 293.0000
5_precision: 0.8151
5_recall: 0.9405
5_f1-score: 0.8702
5_support: 269.0000
6_precision: 0.8878
6_recall: 0.9052
6_f1-score: 0.8963
6_support: 116.0000
7_precision: 0.5780
7_recall: 0.8679
7_f1-score: 0.6784
7_support: 53.0000
8_precision: 0.8708
8_recall: 0.9091
8_f1-score: 0.8895
8_support: 44.0000
accuracy: 0.9919
macro avg_precision: 0.8874
macro avg_recall: 0.9463
macro avg_f1-score: 0.9108
macro avg_support: 31724.0000
weighted avg_precision: 0.9931
weigh


Aggregated Metrics for 4 runs:
test_loss: 0.0332
test_accuracy: 0.9913
balanced_accuracy: 0.9437
roc_auc: 0.9994
pr_auc: 0.9974
f2: 0.9914
mcc: 0.9688
0_precision: 0.9991
0_recall: 0.9961
0_f1-score: 0.9976
0_support: 26862.0000
1_precision: 0.9937
1_recall: 0.9629
1_f1-score: 0.9780
1_support: 2657.0000
2_precision: 0.9470
2_recall: 0.9923
2_f1-score: 0.9689
2_support: 908.0000
3_precision: 0.9449
3_recall: 0.9655
3_f1-score: 0.9550
3_support: 522.0000
4_precision: 0.9266
4_recall: 0.9710
4_f1-score: 0.9480
4_support: 293.0000
5_precision: 0.7981
5_recall: 0.9387
5_f1-score: 0.8601
5_support: 269.0000
6_precision: 0.8835
6_recall: 0.8966
6_f1-score: 0.8899
6_support: 116.0000
7_precision: 0.5454
7_recall: 0.8726
7_f1-score: 0.6575
7_support: 53.0000
8_precision: 0.8642
8_recall: 0.8977
8_f1-score: 0.8806
8_support: 44.0000
accuracy: 0.9913
macro avg_precision: 0.8781
macro avg_recall: 0.9437
macro avg_f1-score: 0.9040
macro avg_support: 31724.0000
weighted avg_precision: 0.9926
weigh


Aggregated Metrics for 5 runs:
test_loss: 0.0353
test_accuracy: 0.9907
balanced_accuracy: 0.9457
roc_auc: 0.9994
pr_auc: 0.9972
f2: 0.9908
mcc: 0.9666
0_precision: 0.9991
0_recall: 0.9958
0_f1-score: 0.9974
0_support: 26862.0000
1_precision: 0.9935
1_recall: 0.9594
1_f1-score: 0.9761
1_support: 2657.0000
2_precision: 0.9450
2_recall: 0.9912
2_f1-score: 0.9674
2_support: 908.0000
3_precision: 0.9431
3_recall: 0.9617
3_f1-score: 0.9522
3_support: 522.0000
4_precision: 0.9158
4_recall: 0.9727
4_f1-score: 0.9429
4_support: 293.0000
5_precision: 0.7841
5_recall: 0.9383
5_f1-score: 0.8520
5_support: 269.0000
6_precision: 0.8656
6_recall: 0.9034
6_f1-score: 0.8833
6_support: 116.0000
7_precision: 0.5259
7_recall: 0.8755
7_f1-score: 0.6450
7_support: 53.0000
8_precision: 0.8634
8_recall: 0.9136
8_f1-score: 0.8874
8_support: 44.0000
accuracy: 0.9907
macro avg_precision: 0.8706
macro avg_recall: 0.9457
macro avg_f1-score: 0.9004
macro avg_support: 31724.0000
weighted avg_precision: 0.9921
weigh